In [58]:
import os
import pandas as pd
import numpy as np
from os import listdir
from os import path
import json
import matplotlib.pyplot as plt
import re
% matplotlib inline

In [76]:
# parse json to files and actions

def parse_json(filepath_list):
    '''
    function used to parse json of each commit json file
    
    Args:
        filepath_list - list of filepaths
    
    Returns:
        files_json - list object contains parsed information
    
    '''
    number_of_files = []
    files_json = []
    
    # each commits
    for path in filepath_list:
        with open(path,'rb') as f:
            data = json.load(f)
            files_list = []
            # each file in commits
            for file in data['files']:
                # parse only cluster file
                for key in file.keys():
                    if re.match('^.*_cluster$',key):
                        actions_list = []
                        actions = file[key]['actions']
                        # each action in file
                        for action in actions:
                            actions_list.append(action['root'])
                        files_list.append(actions_list)
        
        files_json.append(files_list)        
    
    # return
    return files_json

# get file path list
filepath_list = ['tmp_JSON_labeled_commits/' + path for path in listdir('tmp_JSON_labeled_commits/')]
files_json = parse_json(filepath_list)

In [77]:
# getting file threshold
threshold = 0.95
number_of_files = [len(files) for files in files_json]

def get_file_threshold(number_of_files, threshold = 0.95):
    '''
    get padding threshold for files dimension
    
    Args:
        number_of_files - array of the number of files in each commits
        threshold - drop all commits with its the number of files beyond this threshold
    Returns:
        padding threshold - number
    '''
    
    total_files = len(number_of_files)
    number_of_files = np.array(number_of_files)
    bincount = np.bincount(number_of_files)

    sum_file = 0
    for index, item in enumerate(bincount):
        sum_file += item
        #print(index,item)
        #print(sum_file)
        if sum_file > threshold*total_files:
            padding_files_threshold = index
            break
            
    return padding_files_threshold

file_threshold = get_file_threshold(number_of_files, threshold)
print(file_threshold)

22
